In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
%matplotlib inline


In [2]:
dataset = pd.read_csv('data-original.csv')

In [3]:
#replace ? with 1
dataset['Bare Nuclei'] = dataset['Bare Nuclei'].replace(['?'], '1')

In [4]:
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
#train model
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

SVC(random_state=0)

In [8]:
#predict test set results and get accuracy
y_pred = classifier.predict(X_test)
print('Accuracy', (y_test == y_pred).mean())

Accuracy 0.9714285714285714


In [9]:
#get confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("true neatives: {}".format((cm)[0][0]))
print("false positives: {}".format((cm)[0][1]))
print("false negatives: {}".format((cm)[1][0]))
print("true positives: {}".format((cm)[1][1]))
#we have 82 true negatives
#3 false positives
#1 false negative
#54 true positives

[[82  3]
 [ 1 54]]
true neatives: 82
false positives: 3
false negatives: 1
true positives: 54


In [10]:
#compute accuracy with k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
#get mean of accuracies
#.2f means 2 decimals after the comma
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
#get standard deviations from accuracies' list(this values should be small)
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 96.43 %
Standard Deviation: 3.48 %


In [12]:
#we tune 3 parameters: C (regularization), kernels and gamma

#the lower C, the stronger regularization
#regularization helps to reduce overfitting
from sklearn.model_selection import GridSearchCV
#creation of 2 dictionaries: 
#one for linear and one for rbf kernel
#reason: gamma parameter can be only be used with rbf kernel
parameters = [{'C':[0.25, 0.5, 0.75, 1],'kernel':['linear']},
        {'C':[0.25, 0.5, 0.75, 1],'kernel':['rbf'], 
         'gamma':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
#create an object as an instance of the GridSearchCV class
grid_search = GridSearchCV(estimator = classifier,
               param_grid = parameters,
               scoring = 'accuracy',
               #cv means we have 10 train-test-folds (k-fold cross-val)
               cv = 10,
               #if grid search runs on machines, -1 means all processors will be used
               n_jobs = -1)
#connect object 'grid_search' to training set
grid_search.fit(X_train, y_train)
#get best accuracy
#last '_' means get an attribute
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Best Accuracy: 96.60 %
Best Parameters: {'C': 0.5, 'kernel': 'linear'}
